# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# Construct Dictionary According to Handbook

In [2]:
df = pd.read_csv("alldata.csv")

relevant_species = list(df['Species_name'].value_counts().index[:10])
attributes = list(df.columns[9:])

def create_species_dict(*absent_phenophases):
    species_dict = dict(zip(attributes, np.zeros(len(attributes), int)))
    for phenophase in absent_phenophases:
        species_dict[phenophase] = 1
    return species_dict

handbook_dicts = {} # Dict mapping species to attribute dicts. 
# Attribute dicts give absent phenophases in the associated species.
# Manually input absent phenophases from SeasonWatch handbook
handbook_dicts[relevant_species[0]] = create_species_dict('Flowers_open','Fruits_open')
handbook_dicts[relevant_species[1]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[2]] = create_species_dict('Flowers_open', 'Fruits_open')
handbook_dicts[relevant_species[3]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[4]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[5]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[6]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[7]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[8]] = create_species_dict('Flowers_male', 'Flowers_Female', 'Fruits_open')
handbook_dicts[relevant_species[9]] = create_species_dict('Flowers_male', 'Flowers_Female')

# Reconstruct alldata.csv With New Columns Indicating Incorrect -2 Values

In [3]:
incorrect_negative_2_df = df.copy()
for attribute in attributes:
    incorrect_negative_2_df[f"{attribute}_incorrect_-2"] = np.zeros(incorrect_negative_2_df.shape[0], dtype=int)

for species in relevant_species:
    species_df = df[df['Species_name'] == species]
    species_dict = handbook_dicts[species]
    for attribute in attributes:
        if species_dict[attribute] == 0:
            false_positive_idx = species_df.index[species_df[attribute] == -2] # Indices of reports that incorrectly assign -2 values (false positive) to phenophases that DO appear in the species
            incorrect_negative_2_df.loc[false_positive_idx, f"{attribute}_incorrect_-2"] = np.ones(len(false_positive_idx)) # Label false positive reports as 1 for the phenophase they incorrectly report.
        if species_dict[attribute] == 1:
            false_negative_idx = species_df.index[species_df[attribute] != -2] # Indices of reports that incorrectly assign values other than -2 (false positive) to phenophases that DO NOT appear in the species
            incorrect_negative_2_df.loc[false_negative_idx, f"{attribute}_incorrect_-2"] = 2*np.ones(len(false_negative_idx)) # Label false negative reports as 2 for the phenophase they incorrectly report.

# Preview CSV File

In [4]:
incorrect_negative_2_df

,Unnamed: 0,Observation_ID,Date_of_observation,User_id,User_Tree_id,Species_name,Lat,Long,State_name,Leaves_fresh,Leaves_mature,Leaves_old,Flowers_bud,Flowers_open,Flowers_male,Flowers_Female,Fruits_unripe,Fruits_ripe,Fruits_open,Leaves_fresh_incorrect_-2,Leaves_mature_incorrect_-2,Leaves_old_incorrect_-2,Flowers_bud_incorrect_-2,Flowers_open_incorrect_-2,Flowers_male_incorrect_-2,Flowers_Female_incorrect_-2,Fruits_unripe_incorrect_-2,Fruits_ripe_incorrect_-2,Fruits_open_incorrect_-2
0,1,388564.0,2020-01-01,20396.0,84299.0,Indian Almond- Terminalia catappa,12.15386,75.22397,Kerala,2.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0,0,0,0,0,0,0,0,0,0
1,2,388565.0,2020-01-01,20396.0,84300.0,Indian Almond- Terminalia catappa,12.15386,75.22397,Kerala,2.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0,0,0,0,0,0,0,0,0,0
2,3,388566.0,2020-01-01,20396.0,84301.0,Fish-tail palm- Caryota urens,12.14060,75.22145,Kerala,0.0,2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0,0,0,0,0,0,0,0,0,0
3,4,388567.0,2020-01-01,20396.0,84302.0,Mast Tree-Monoon longifolium,12.14060,75.22145,Kerala,1.0,2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0,0,0,0,0,0,0,0,0,0
4,5,388568.0,2020-01-01,20396.0,84303.0,Indian Almond- Terminalia catappa,12.14060,75.22145,Kerala,0.0,1.0,2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592588,592589,736977.0,2023-12-04,26471.0,124684.0,Jackfruit- Artocarpus heterophyllus,8.53024,77.04296,Kerala,1.0,2.0,1.0,0.0,-2.0,0.0,0.0,0.0,0.0,-2.0,0,0,0,0,0,0,0,0,0,0
592589,592590,736978.0,2023-12-12,26471.0,124684.0,Jackfruit- Artocarpus heterophyllus,8.53024,77.04296,Kerala,1.0,2.0,1.0,0.0,-2.0,1.0,1.0,0.0,0.0,-2.0,0,0,0,0,0,0,0,0,0,0
592590,592591,736979.0,2023-12-20,26471.0,124684.0,Jackfruit- Artocarpus heterophyllus,8.53024,77.04296,Kerala,1.0,2.0,1.0,1.0,-2.0,1.0,1.0,0.0,0.0,-2.0,0,0,0,0,0,0,0,0,0,0
592591,592592,736981.0,2023-12-28,26471.0,124684.0,Jackfruit- Artocarpus heterophyllus,8.53024,77.04296,Kerala,1.0,2.0,1.0,1.0,-2.0,1.0,1.0,0.0,0.0,-2.0,0,0,0,0,0,0,0,0,0,0


# Test

In [5]:
# For each species, check if reports for each phenophase is marked appropriately in the incorrect_-2 column.
for species in relevant_species:
    print(f'-----------------------------------\n{species}\n-----------------------------------')
    for attribute in attributes:
        a = incorrect_negative_2_df[incorrect_negative_2_df['Species_name'] == species][[attribute, f'{attribute}_incorrect_-2']]
        #display(a[a[attribute] != -2].drop_duplicates())
        display(a.drop_duplicates())

-----------------------------------
Jackfruit- Artocarpus heterophyllus
-----------------------------------


,Leaves_fresh,Leaves_fresh_incorrect_-2
13,1.0,0
25,0.0,0
28,2.0,0
37,-1.0,0
104179,NaN,0
179774,-2.0,1


,Leaves_mature,Leaves_mature_incorrect_-2
13,2.0,0
17,1.0,0
71,0.0,0
23370,-1.0,0
99511,NaN,0
187390,-2.0,1


,Leaves_old,Leaves_old_incorrect_-2
13,1.0,0
25,0.0,0
28,NaN,0
52,2.0,0
208,-2.0,1
171855,-1.0,0


,Flowers_bud,Flowers_bud_incorrect_-2
13,1.0,0
28,0.0,0
177,2.0,0
208,-2.0,1
512,-1.0,0
103952,NaN,0


...

,Flowers_male,Flowers_male_incorrect_-2
14,-2.0,0
142,NaN,2
419,0.0,2


,Flowers_Female,Flowers_Female_incorrect_-2
14,-2.0,0
142,NaN,2
419,0.0,2


,Fruits_unripe,Fruits_unripe_incorrect_-2
14,1.0,0
142,0.0,0
188,2.0,0
37449,-1.0,0
170098,-2.0,1


,Fruits_ripe,Fruits_ripe_incorrect_-2
14,2.0,0
81,1.0,0
142,0.0,0
27710,-1.0,0
170098,-2.0,1


,Fruits_open,Fruits_open_incorrect_-2
14,1.0,0
142,NaN,0
188,0.0,0
419,2.0,0
178356,-1.0,0
185247,-2.0,1


# Download CSV File

In [6]:
incorrect_negative_2_df.to_csv('alldata_labeling_-2.csv', index=False)